In [ ]:
from db.connection import get_engine
import pandas as pd

engine = get_engine()
pd.read_sql("""
    SELECT table_schema, table_name
    FROM information_schema.tables
    WHERE table_schema NOT IN ('pg_catalog', 'information_schema')
    ORDER BY table_schema, table_name
""", engine)


,table_schema,table_name
0,legacy,league_gamelogs
1,legacy,player_boxscores_traditional_v3
2,raw,box_score_traditional_v3


In [24]:
import pandas as pd

from nba_api.stats.endpoints import (
    BoxScoreTraditionalV3,
    LeagueGameLog
)

GAME_ID = "0022500001"  # replace if needed
SEASON = "2025-26"


In [25]:
bs = BoxScoreTraditionalV3(game_id=GAME_ID)

team_bs_df = bs.get_data_frames()[2]
team_bs_df.head()


,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,minutes,fieldGoalsMade,fieldGoalsAttempted,fieldGoalsPercentage,...,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints
0,0022500001,1610612760,Oklahoma City,Thunder,OKC,thunder,290:00,46,104,0.442,...,11,27,38,29,12,4,11,27,125,1.0
1,0022500001,1610612745,Houston,Rockets,HOU,rockets,290:00,43,97,0.443,...,16,36,52,23,6,5,22,26,124,-1.0


In [26]:
team_bs_df.columns = (
    team_bs_df.columns
    .str.replace("([a-z0-9])([A-Z])", r"\1_\2", regex=True)
    .str.lower()
)

team_bs_df


,game_id,team_id,team_city,team_name,team_tricode,team_slug,minutes,field_goals_made,field_goals_attempted,field_goals_percentage,...,rebounds_offensive,rebounds_defensive,rebounds_total,assists,steals,blocks,turnovers,fouls_personal,points,plus_minus_points
0,0022500001,1610612760,Oklahoma City,Thunder,OKC,thunder,290:00,46,104,0.442,...,11,27,38,29,12,4,11,27,125,1.0
1,0022500001,1610612745,Houston,Rockets,HOU,rockets,290:00,43,97,0.443,...,16,36,52,23,6,5,22,26,124,-1.0


In [27]:
lg = LeagueGameLog(
    season=SEASON,
    season_type_all_star="Regular Season"
)

lg_df = lg.get_data_frames()[0]
lg_df.head()


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22025,1610612744,GSW,Golden State Warriors,0022500002,2025-10-21,GSW @ LAL,W,240,38,...,31,40,29,10,4,19,27,119,10,1
1,22025,1610612747,LAL,Los Angeles Lakers,0022500002,2025-10-21,LAL vs. GSW,L,240,42,...,32,39,23,7,2,20,21,109,-10,1
2,22025,1610612745,HOU,Houston Rockets,0022500001,2025-10-21,HOU @ OKC,L,290,43,...,36,52,23,6,5,25,26,124,-1,1
3,22025,1610612760,OKC,Oklahoma City Thunder,0022500001,2025-10-21,OKC vs. HOU,W,290,46,...,27,38,29,12,4,12,27,125,1,1
4,22025,1610612738,BOS,Boston Celtics,0022500083,2025-10-22,BOS vs. PHI,L,240,41,...,32,42,16,7,4,11,28,116,-1,1


In [28]:
lg_game_df = lg_df[lg_df["GAME_ID"] == GAME_ID].copy()
lg_game_df


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
2,22025,1610612745,HOU,Houston Rockets,0022500001,2025-10-21,HOU @ OKC,L,290,43,...,36,52,23,6,5,25,26,124,-1,1
3,22025,1610612760,OKC,Oklahoma City Thunder,0022500001,2025-10-21,OKC vs. HOU,W,290,46,...,27,38,29,12,4,12,27,125,1,1


In [29]:
bs_team = team_bs_df[[
    "game_id",
    "team_id",
    "points",
    "rebounds_total",
    "assists",
    "turnovers",
    "field_goals_made",
    "field_goals_attempted",
    "free_throws_made",
    "free_throws_attempted"
]].copy()

bs_team


,game_id,team_id,points,rebounds_total,assists,turnovers,field_goals_made,field_goals_attempted,free_throws_made,free_throws_attempted
0,0022500001,1610612760,125,38,29,11,46,104,20,25
1,0022500001,1610612745,124,52,23,22,43,97,27,31


In [30]:
lg_team = lg_game_df[[
    "GAME_ID",
    "TEAM_ID",
    "PTS",
    "REB",
    "AST",
    "TOV",
    "FGM",
    "FGA",
    "FTM",
    "FTA"
]].copy()

lg_team.columns = [
    "game_id",
    "team_id",
    "points",
    "rebounds_total",
    "assists",
    "turnovers",
    "field_goals_made",
    "field_goals_attempted",
    "free_throws_made",
    "free_throws_attempted"
]

lg_team


,game_id,team_id,points,rebounds_total,assists,turnovers,field_goals_made,field_goals_attempted,free_throws_made,free_throws_attempted
2,0022500001,1610612745,124,52,23,25,43,97,27,31
3,0022500001,1610612760,125,38,29,12,46,104,20,25


In [31]:
comparison = bs_team.merge(
    lg_team,
    on=["game_id", "team_id"],
    suffixes=("_boxscore", "_league")
)

comparison


,game_id,team_id,points_boxscore,rebounds_total_boxscore,assists_boxscore,turnovers_boxscore,field_goals_made_boxscore,field_goals_attempted_boxscore,free_throws_made_boxscore,free_throws_attempted_boxscore,points_league,rebounds_total_league,assists_league,turnovers_league,field_goals_made_league,field_goals_attempted_league,free_throws_made_league,free_throws_attempted_league
0,0022500001,1610612760,125,38,29,11,46,104,20,25,125,38,29,12,46,104,20,25
1,0022500001,1610612745,124,52,23,22,43,97,27,31,124,52,23,25,43,97,27,31


In [32]:
diff_cols = {}

for col in [
    "points",
    "rebounds_total",
    "assists",
    "turnovers",
    "field_goals_made",
    "field_goals_attempted",
    "free_throws_made",
    "free_throws_attempted"
]:
    diff_cols[col] = (
        comparison[f"{col}_boxscore"]
        - comparison[f"{col}_league"]
    )

diff_df = pd.DataFrame(diff_cols)
diff_df


,points,rebounds_total,assists,turnovers,field_goals_made,field_goals_attempted,free_throws_made,free_throws_attempted
0,0,0,0,-1,0,0,0,0
1,0,0,0,-3,0,0,0,0


In [35]:
team_bs_df.columns

Index(['game_id', 'team_id', 'team_city', 'team_name', 'team_tricode',
       'team_slug', 'minutes', 'field_goals_made', 'field_goals_attempted',
       'field_goals_percentage', 'three_pointers_made',
       'three_pointers_attempted', 'three_pointers_percentage',
       'free_throws_made', 'free_throws_attempted', 'free_throws_percentage',
       'rebounds_offensive', 'rebounds_defensive', 'rebounds_total', 'assists',
       'steals', 'blocks', 'turnovers', 'fouls_personal', 'points',
       'plus_minus_points'],
      dtype='object')

In [37]:
lg_df.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')

In [38]:
import pandas as pd
from nba_api.stats.endpoints import LeagueGameLog

SEASON = "2025-26"

lg = LeagueGameLog(
    season=SEASON,
    season_type_all_star="Regular Season"
)

lg_df = lg.get_data_frames()[0]

inspection = pd.DataFrame({
    "column": lg_df.columns,
    "pandas_dtype": lg_df.dtypes.astype(str),
    "example_value": [
        lg_df[col].dropna().iloc[0] if lg_df[col].notna().any() else None
        for col in lg_df.columns
    ]
})

inspection


,column,pandas_dtype,example_value
SEASON_ID,SEASON_ID,object,22025
TEAM_ID,TEAM_ID,int64,1610612744
TEAM_ABBREVIATION,TEAM_ABBREVIATION,object,GSW
TEAM_NAME,TEAM_NAME,object,Golden State Warriors
GAME_ID,GAME_ID,object,0022500002
GAME_DATE,GAME_DATE,object,2025-10-21
MATCHUP,MATCHUP,object,GSW @ LAL
WL,WL,object,W
MIN,MIN,int64,240
FGM,FGM,int64,38


In [39]:
missing = lg_df[lg_df["FG3M"].isna()]
missing.head()


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
